## Load Dataset From Huggingface

In [5]:
import datasets as dts

HF_TOKEN=""

dataset = dts.load_dataset('squad_v1_pt')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

## Normalize Dataset

- Use the format proposed by [lmqg/qg_squad](https://huggingface.co/datasets/lmqg/qg_squad)

- Replace HTML codecs for real strings



In [2]:
dataset['train']['context'][:3]

['Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda &quot;Venite Ad Me Omnes&quot;. Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.',
 'Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços er

In [3]:
answer = dataset["train"]["answers"][2]

print(answer)
answer_len = len(answer["text"][0])
ctx = dataset['train']['context'][0]
answer_start = answer["answer_start"][0]
ctx = list(ctx)
ctx.insert(answer_start, "<h1>")
print("".join(ctx))

{'text': ['o edifício principal'], 'answer_start': [81]}
Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro d<h1>o edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda &quot;Venite Ad Me Omnes&quot;. Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.


In [4]:
from html.entities import name2codepoint
import re

REGEX_PATTERN = re.compile(r'(?<=\&)\w+(?=\;)')

def replace_htmlcodecs(text):

    for match_obj in REGEX_PATTERN.finditer(text):

        str_pattern = match_obj.group()
        text = re.sub(f'\&{str_pattern}\;', chr(name2codepoint[str_pattern]), text)

    return text

In [6]:
def preprocess_dataset(example):

    question = replace_htmlcodecs(example["question"])
    context = replace_htmlcodecs(example['context'])
    answer_start = example["answers"]["answer_start"][0]
    answer_text =  replace_htmlcodecs(example["answers"]["text"][0])
    answer_len = len(answer_text)
    answer_ctx = list(example['context'])
    
    example["paragraph_id"] =  example["id"]
    example['paragraph'] = context
    example["question"] =  question
    example["answer"] = answer_text
    example["paragraph_question"] = f"question: {question} paragraph: {context}"
    example["is_valid"] = answer_start != 0 and (example['context'][answer_start:answer_start+answer_len] == answer_text)

    #- **Paragraph Answer**: The paragraph with the answer
    # highlighted with `<h1> <h1>` tags. - Done
    if example["is_valid"]:
       
        
        answer_ctx.insert(answer_start, " <h1> ")
        answer_ctx.insert(answer_start + answer_len + 1, " <h1> ")
        answer_ctx =  "".join(answer_ctx)
        example["paragraph_answer"] = replace_htmlcodecs(answer_ctx)
    else:
        example["paragraph_answer"] = ""
    return example

In [ ]:
processed_dataset = dataset.map(preprocess_dataset,
                                remove_columns=['answers', 'title', 'context', 'id']).filter(lambda ex: ex["is_valid"])


In [ ]:
processed_dataset

In [188]:
example = processed_dataset['train'][16]

context = example['paragraph']
question =  example['question']
answer =  example['answer']
paragraph_question =  example['paragraph_question']
paragraph_answer_text =  example['paragraph_answer']

print("Context: ", context)
print("Question: ", question)
print("Answer: ", answer)
print("paragraph_question: ", paragraph_question)
print("paragraph_answer: ", paragraph_answer_text)

Context:  A universidade primeiro ofereceu pós-graduação, sob a forma de um Master of Arts (MA), no ano lectivo de 1854-1855. O programa expandiu-se para incluir o Mestrado em Direito (LL.M.) e o Mestrado em Engenharia Civil nos seus primeiros estágios de crescimento, antes que uma educação formal de pós-graduação fosse desenvolvida com uma tese não requerida para receber os diplomas. Isso mudou em 1924, com requisitos formais desenvolvidos para pós-graduação, incluindo a oferta de doutorado (PhD). Hoje, cada uma das cinco faculdades oferece educação de pós-graduação. A maioria dos departamentos da Faculdade de Artes e Letras oferece programas de doutorado, enquanto um programa profissional de Mestrado em Divindade (M.Div.) Também existe. Todos os departamentos da Faculdade de Ciências oferecem programas de doutorado, exceto o Departamento de Estudos Pré-Profissionais. A Escola de Arquitetura oferece um Mestrado em Arquitetura, enquanto cada um dos departamentos da Faculdade de Engenha

In [189]:
for split in processed_dataset.keys():
    processed_dataset[split].to_json(f'../datasets/qg_squad_v1_pt/{split}.json')

Creating json from Arrow format: 100%|██████████| 7/7 [00:00<00:00, 15.56ba/s]


In [11]:
#from google.colab import userdata

processed_dataset.push_to_hub('qg_squad_v1_pt', token=HF_TOKEN)

NameError: name 'processed_dataset' is not defined

## Extract extra information from the dataset using Spacy


- **Sentence**: Sentence where the answer is found. - Done

- **Sentence Answer:** Sentence where the answer is found with the answer highlighted with `<h1> <h1>` tags. - Done

- **Paragraph Answer**: The paragraph with the answer highlighted with `<h1> <h1>` tags. - Done

- **Paragraph Sentence**: The paragraph with the sentence highlighted with `<h1> <h1>` tags. - Done


### Spacy setup

- Functions and components definitions

- Attributed definitions


In [202]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Token, Doc
from spacy.language import Language


In [203]:
def get_sentence(token):
    """
     Gets the sentence where the answer is found.
    """

    for sent in token.doc.sents:
        if sent.start <= token.i <= sent.end:
            return sent

In [204]:
if not Doc.has_extension("questions"):
    Doc.set_extension("questions", default=[])

if not Doc.has_extension("answers"):
    Doc.set_extension("answers", default=[])

if not Doc.has_extension("parag_ans"):
    Doc.set_extension("parag_ans", default=[])

if not Doc.has_extension("answer_sentence"):
    Doc.set_extension("answer_sentence", default=[])

if not Doc.has_extension("sentences"):
    Doc.set_extension("sentences", default=[])

if not Doc.has_extension("paragraph_sentence"):
    Doc.set_extension("paragraph_sentence", default=[])

if not Token.has_extension("is_delimiter"):
    Token.set_extension("is_delimiter", default=False, force=True)

if not Token.has_extension("sent"):
    Token.set_extension('sent', getter=get_sentence, force=True)

In [205]:
patterns = [[{"TEXT": {"REGEX": "<.*?>"}},],
[{"TEXT": {"REGEX": "<"}},
{"TEXT": {"REGEX": "h1>"}}]
]
nlp = spacy.blank('pt')
matcher = Matcher(nlp.vocab)
matcher.add("delimiter_token", patterns)

In [206]:
def spacy_get_text_boundaries(text, context_doc):
    """
    Find the boundaries of the answers tokens in the context document
    text: string # answer in string format
    context_doc: spacy document # context where the answers boundaries must
                                #  be found
    """
    matcher = Matcher(nlp.vocab)

    pattern = [[{"ORTH": token.text} ]  for token in nlp(text)]
    matcher.add(text, pattern)
    matches = matcher(context_doc)
   
    answers_starts = []
    answers_ends = []
    for _, start, end in matches:
        answers_starts.append(start)
        answers_ends.append(end)

    return min(answers_starts), max(answers_ends)



        
def get_answer_sentence(token):
    """
     Gets the sentence where the answer is found.
    """

    for sent in token.doc.sents:
        if token._.is_delimiter:
            return sent

def remove_h1_token(text_doc):
    context_words = [token.text for token in text_doc if token.text != "<h1>"]

    spaces = [not token.is_punct for token in text_doc if token.text != "<h1>"]
    #start_span, end_span = start_span - num_h1, end_span - num_h1
    spaces.pop(0)
    spaces += [False]
    return Doc(nlp.vocab, words=context_words, spaces=spaces)

def spacy_mark_text_span(text_doc, start_span, end_span,):
  context_words = [token.text for token in text_doc if token.text != "<h1>"]
  num_h1 = (len(text_doc)) - len(context_words)
  spaces = [not token.is_punct for token in text_doc if token.text != "<h1>"]
  #start_span, end_span = start_span - num_h1, end_span - num_h1
  spaces.pop(0)
  spaces += [False]
  
  #FIX: punctuations are one index after where it should be
  spaces.insert(start_span,False)
  spaces.insert(end_span,False)
  
  context_words.insert(start_span, "<h1>")
  context_words.insert(end_span-num_h1, "<h1>")
 
  return Doc(nlp.vocab, words=context_words, spaces=spaces)

In [207]:
@Language.component("delimiter_tokenizer")
def delimiter_tokenizer(doc):
    matches = matcher(doc)
  
    for match_id, start, end in matches:
        for i in range(start, end):
            token = doc[i]
            token._.is_delimiter = True
            # Process the token if needed
            # For example, you can set a custom attribute on the token
        matched_span = doc[start: end] 
      
        with doc.retokenize() as retokenizer:
            retokenizer.merge(matched_span)
    return doc

@Language.component("answer_sentence")
def answer_sentence(doc):
    """
    Find the answer inside the paragragh using Ruled Based Matcher
    """

    answer_text = ""
    ans_found = False
    parag_sentence = None
    for token in doc:
        if ans_found and token.text != "<h1>":
            answer_text += token.text + " "
            
        if token._.is_delimiter :   
            
            
            if remove_h1_token(token._.sent) not in doc._.sentences:
                ans_found = True
                #- **Sentence Answer:** Sentence where the answer is found with the answer highlighted 
                #with `<h1> <h1>` tags. - Done
                doc._.answer_sentence.append(token._.sent)

                # **Sentence Answer:** Sentence where the answer 
                # is found with the answer highlighted with `<h1> <h1>` tags.
                doc._.sentences.append(remove_h1_token(token._.sent))

                #- **Paragraph Sentence**: The paragraph with the sentence
                # highlighted with `<h1> <h1>` tags. - DOne
                parag_sentence = spacy_mark_text_span(doc, 
                                                            token._.sent.start,
                                                            token._.sent.end)
                doc._.paragraph_sentence.append(parag_sentence)
            else:
                ans_found = False
                answer_text = ""
               
    return doc


In [208]:
# Add a computed property, which will be accessible as token._.sent
nlp.tokenizer.add_special_case("<h1>", [{"ORTH": "<h1>"}])

if "delimiter_tokenizer" not in nlp.pipe_names:
    nlp.add_pipe("delimiter_tokenizer", name="delimiter_tokenizer")
if 'sentencizer' not in nlp.pipe_names:
    nlp.add_pipe('sentencizer')
if "answer_sentence" not in nlp.pipe_names:
    nlp.add_pipe("answer_sentence", name="answer_sentence")

In [209]:
doc = nlp(paragraph_answer_text)
print(doc._.answer_sentence)
print(doc._.sentences)
print(doc._.paragraph_sentence)

[A universidade primeiro ofereceu pós-graduação, sob a forma de um Master of Arts (MA), no ano lectivo de  <h1> 1854 <h1> -1855., A universidade primeiro ofereceu pós-graduação, sob a forma de um Master of Arts (MA), no ano lectivo de  <h1> 1854 <h1> -1855.]
[A universidade primeiro ofereceu pós-graduação, sob a forma de um Master of Arts( MA), no ano lectivo de   1854 -1855., A universidade primeiro ofereceu pós-graduação, sob a forma de um Master of Arts( MA), no ano lectivo de   1854 -1855.]
[<h1>A universidade primeiro ofereceu pós-graduação, sob a forma de um Master of Arts( MA), no ano lectivo de   1854 -1855<h1> . Oprograma expandiu-se para incluir o Mestrado em Direito( LL.M.) e o Mestrado em Engenharia Civil nos seus primeiros estágios de crescimento, antes que uma educação formal de pós-graduação fosse desenvolvida com uma tese não requerida para receber os diplomas. Isso mudou em 1924, com requisitos formais desenvolvidos para pós-graduação, incluindo a oferta de doutorado( 

In [210]:
def preprocess_spacy(example):
    doc = nlp(example["paragraph_answer"])
    
    example["sentence"] = doc._.sentences[0].text
    example["answer_sentence"] = doc._.answer_sentence[0].text
    example["paragraph_sentence"] = doc._.paragraph_sentence[0].text
    return example

In [211]:
processed_dataset_spacy = processed_dataset.map(preprocess_spacy)
processed_dataset_spacy

Map:   0%|          | 0/52135 [00:00<?, ? examples/s]

Map: 100%|██████████| 6378/6378 [00:31<00:00, 200.90 examples/s]


DatasetDict({
    train: Dataset({
        features: ['question', 'paragraph_id', 'paragraph', 'answer', 'paragraph_question', 'is_valid', 'paragraph_answer', 'sentence', 'answer_sentence', 'paragraph_sentence'],
        num_rows: 52135
    })
    validation: Dataset({
        features: ['question', 'paragraph_id', 'paragraph', 'answer', 'paragraph_question', 'is_valid', 'paragraph_answer', 'sentence', 'answer_sentence', 'paragraph_sentence'],
        num_rows: 6378
    })
})

In [212]:
#from google.colab import userdata

processed_dataset.push_to_hub('qg_squad_v1_pt', token=HF_TOKEN)

README.md: 100%|██████████| 669/669 [00:00<00:00, 2.06MB/s]
